# 2. Advanced Tutorials - 1. Customization - 2. Customer layers

URL : https://www.tensorflow.org/beta/tutorials/eager/custom_layers

We recommend using tf.keras as a high-level API for building neural networks. <br>
That said, most TensorFlow APIs are usable with eager execution.<br>

### 목차
1. Layers:common sets of useful operations
2. Implementing custom layers
3. Models:composing layers

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


# 1. Layers:common sets of useful operations

TensorFlow includes the full Keras API in the tf.keras package, and the Keras layers are very useful when building your own models.

In [3]:
#Most layers take as a first argument the number of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary
layer = tf.keras.layers.Dense(10, input_shape=(None,5))

The full list of pre-existing layers can be seen in the documentation. <br>
It includes Dense (a fully-connected layer), Conv2D, LSTM, BatchNormalization, Dropout, and many others.

In [4]:
layer(tf.zeros([10,5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.2280271 , -0.60855603,  0.15571153, -0.40853864, -0.01158875,
          0.31229687,  0.4122607 , -0.2299824 ,  0.6106041 ,  0.1563198 ],
        [ 0.07169735, -0.512296  , -0.21175817,  0.46712393,  0.37026936,
          0.22550094,  0.5880677 ,  0.33798647, -0.44650972, -0.01501089],
        [ 0.30120134, -0.4717366 ,  0.50412863,  0.29858047,  0.3203612 ,
          0.15965658,  0.48445624, -0.41152123,  0.0111109 , -0.18924671],
        [ 0.4724663 ,  0.02966458, -0.29876322, -0.5243287 ,  0.3432626 ,
          0.40037566, -0.09515655,  0.37963063,  0.04484695, -0.3192841 ],
        [-0.30729488,  0.48230678, -0.12428796,  0.31491196,  0.23150897,
          0.31834316,  0.10172778, -0.0533343 , -0.47070235, -0.42248136]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
layer.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.2280271 , -0.60855603,  0.15571153, -0.40853864, -0.01158875,
          0.31229687,  0.4122607 , -0.2299824 ,  0.6106041 ,  0.1563198 ],
        [ 0.07169735, -0.512296  , -0.21175817,  0.46712393,  0.37026936,
          0.22550094,  0.5880677 ,  0.33798647, -0.44650972, -0.01501089],
        [ 0.30120134, -0.4717366 ,  0.50412863,  0.29858047,  0.3203612 ,
          0.15965658,  0.48445624, -0.41152123,  0.0111109 , -0.18924671],
        [ 0.4724663 ,  0.02966458, -0.29876322, -0.5243287 ,  0.3432626 ,
          0.40037566, -0.09515655,  0.37963063,  0.04484695, -0.3192841 ],
        [-0.30729488,  0.48230678, -0.12428796,  0.31491196,  0.23150897,
          0.31834316,  0.10172778, -0.0533343 , -0.47070235, -0.42248136]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [7]:
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[ 0.2280271 , -0.60855603,  0.15571153, -0.40853864, -0.01158875,
          0.31229687,  0.4122607 , -0.2299824 ,  0.6106041 ,  0.1563198 ],
        [ 0.07169735, -0.512296  , -0.21175817,  0.46712393,  0.37026936,
          0.22550094,  0.5880677 ,  0.33798647, -0.44650972, -0.01501089],
        [ 0.30120134, -0.4717366 ,  0.50412863,  0.29858047,  0.3203612 ,
          0.15965658,  0.48445624, -0.41152123,  0.0111109 , -0.18924671],
        [ 0.4724663 ,  0.02966458, -0.29876322, -0.5243287 ,  0.3432626 ,
          0.40037566, -0.09515655,  0.37963063,  0.04484695, -0.3192841 ],
        [-0.30729488,  0.48230678, -0.12428796,  0.31491196,  0.23150897,
          0.31834316,  0.10172778, -0.0533343 , -0.47070235, -0.42248136]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

# 2. Implementing custom layers

The best way to implement your own layer is extending the tf.keras.Layer class and implementing:
- \* __\_\_init\_\___ , where you can do all input-independent initialization 
- \* __build__, where you know the shapes of the input tensors and can do the rest of the initialization 
- \* __call__, where you do the forward computation

Note that you don't have to wait until build is called to create your variables, you can also create them in \_\_init\_\_. <br>
However, the advantage of creating them in build is that it enables late variable creation based on the shape of the inputs the layer will operate on. <br>
=> enables late variable creation? 뭔 말<br>
On the other hand, creating variables in __init__ would mean that shapes required to create the variables will need to be explicitly specified.<br>

In [8]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs
        
    def build(self, input_shape):
        self.kernel = self.add_variable('kernel', shape=[int(input_shape[-1]), self.num_outputs])
        
    def call(self, input):
        return tf.matmul(input, self.kernel)
    
layer = MyDenseLayer(10)
print(layer(tf.zeros([10,5])))
print(layer.trainable_variables)

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]], shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32, numpy=
array([[-0.31830022,  0.41905528,  0.0314694 , -0.5025718 , -0.07129866,
        -0.3728037 ,  0.09434938, -0.27421716,  0.13896817,  0.07145447],
       [-0.12444615,  0.07562363, -0.01160127, -0.3638782 , -0.15126851,
        -0.10554862,  0.34031224, -0.5281734 , -0.05240583,  0.33716607],
       [ 0.41005737, -0.45011872,  0.4481482 ,  0.1853888 ,  0.5341503 ,
         0.24707466,  0.3368954 ,  0.36519325, -0.17926222,  0.0956136 ],
       [-0.28885728, -0.42473686, -0.28443643,  0.1492737 , -0.44854555,
         0.59450597, -0.3328274 , -0

# 3. Models:composing layers

In [9]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__()
        filters1, filters2, filters3 = filters
        
        self.conv2a = tf.keras.layers.Conv2D(filters1, (1,1))
        self.bn2a = tf.keras.layers.BatchNormalization()
        
        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()
        
        self.conv2c = tf.keras.layers.Conv2D(filters3, (1,1))
        self.bn2c = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)
        
        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)
        
        x += input_tensor
        return tf.nn.relu(x)

In [10]:
block = ResnetIdentityBlock(1, [1,2,3])
print(block(tf.zeros([1,2,3,3])))

tf.Tensor(
[[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]], shape=(1, 2, 3, 3), dtype=float32)


In [11]:
a = [x.name for x in block.trainable_variables]
print(a)
print(len(a))

['resnet_identity_block/conv2d/kernel:0', 'resnet_identity_block/conv2d/bias:0', 'resnet_identity_block/batch_normalization_v2/gamma:0', 'resnet_identity_block/batch_normalization_v2/beta:0', 'resnet_identity_block/conv2d_1/kernel:0', 'resnet_identity_block/conv2d_1/bias:0', 'resnet_identity_block/batch_normalization_v2_1/gamma:0', 'resnet_identity_block/batch_normalization_v2_1/beta:0', 'resnet_identity_block/conv2d_2/kernel:0', 'resnet_identity_block/conv2d_2/bias:0', 'resnet_identity_block/batch_normalization_v2_2/gamma:0', 'resnet_identity_block/batch_normalization_v2_2/beta:0']
12


Much of the time, however, models which compose many layers simply call one layer after the other. <br>
This can be done in very little code using tf.keras.Sequential

In [12]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1,1), input_shape=(None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1,2,3,3]))

<tf.Tensor: id=737, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>